In [37]:
import ipywidgets as widgets
from IPython.display import display
import functions as functions
import importlib
import warnings
import pathlib
import pandas as pd
import dropbox
from dropbox.exceptions import AuthError
import os
warnings.filterwarnings('ignore')


# Event Field
## Automated Event File Generation
### by Veering Analytics

#### Please enter dropbox details and select files to upload to kernel

In [38]:
db_token_widget = widgets.Text(description='Dropbox Access Token')
display(db_token_widget)

Text(value='', description='Dropbox Access Token')

In [39]:

def dropbox_connect(at):
    """Create a connection to Dropbox."""

    try:
        dbx = dropbox.Dropbox(at)
    except AuthError as e:
        print('Error connecting to Dropbox with access token: ' + str(e))
    return dbx

def dropbox_list_files(path,at):
    """Return a Pandas dataframe of files in a given Dropbox folder path in the Apps directory.
    """

    dbx = dropbox_connect(at)

    try:
        files = dbx.files_list_folder(path).entries
        files_list = []
        for file in files:
            if isinstance(file, dropbox.files.FileMetadata):
                metadata = {
                    'name': file.name,
                    'path_display': file.path_display,
                    'client_modified': file.client_modified,
                    'server_modified': file.server_modified
                }
                files_list.append(metadata)

        df = pd.DataFrame.from_records(files_list)
        return df.sort_values(by='server_modified', ascending=False)

    except Exception as e:
        print('Error getting list of files from Dropbox: ' + str(e))
        



def dropbox_download_file(dropbox_file_path, local_file_path,at):
    """Download a file from Dropbox to the local machine."""

    try:
        dbx = dropbox_connect(at)

        with open(local_file_path, 'wb') as f:
            metadata, result = dbx.files_download(path=dropbox_file_path)
            f.write(result.content)
    except Exception as e:
        print('Error downloading file from Dropbox: ' + str(e))



In [40]:
def get_db_files(b=None):
    
    global at
    at = db_token_widget.value
    

    path_log = '/eventField_test/logs'
    path_config = '/eventField_test/configs'

    log_files = dropbox_list_files(path_log,at)
    config_files = dropbox_list_files(path_config,at)

    dropbox_download_file(log_files.path_display[1], str(os.getcwd())+'/test.csv',at)

In [41]:
db_button = widgets.Button(description='Load DB Files')
db_button.on_click(get_db_files)
display(db_button)

Button(description='Load DB Files', style=ButtonStyle())

In [42]:
def load2(b=None):
    use_config = top_pane_dict['buttons'][0].value
    log_path = top_pane_dict['file_choosers'][0].selected
    config_path = top_pane_dict['file_choosers'][1].selected
    
    try:
        log, var_dict, startDict, raceIndex, marksDict, raceEvents, eventLog, selector_dict_day = functions.load_and_process(log_path,config_path,use_config)
        load2.log = log
        load2.var_dict = var_dict
        load2.startDict = startDict
        load2.raceIndex = raceIndex
        load2.marksDict = marksDict
        load2.raceEvents = raceEvents
        load2.eventLog = eventLog
        load2.selector_dict_day = selector_dict_day
        plots = functions.plot_race(startDict, raceEvents, log, marksDict, raceIndex) # create plots for each race
        plot_tab = functions.build_plot_tabs(plots)
        mid_pane = widgets.VBox([plot_tab,selector_dict_day['formatted']])
        load2.mid_pane = mid_pane
        
    except:
        load2.mid_pane = widgets.Label("Error Loading Files")
    



####  Define log and config files

In [45]:
top_pane_dict = functions.build_top_pane()
display(top_pane_dict['formatted'][0])

In [46]:
load_button = widgets.Button(description='Load and Process Files')
mid_pane_output = widgets.Output()

    
@mid_pane_output.capture()
def load(change):
    load2()
    mid_pane_output.clear_output()
    with mid_pane_output:
        display(load2.mid_pane)
    
    
load_button.on_click(load)
display(load_button)
mid_pane_output

Button(description='Load and Process Files', style=ButtonStyle())

Output()

In [47]:
def dropbox_upload_file(local_path, local_file, dropbox_file_path,at):
    """Upload a file from the local machine to a path in the Dropbox app directory.

    Args:
        local_path (str): The path to the local file.
        local_file (str): The name of the local file.
        dropbox_file_path (str): The path to the file in the Dropbox app directory.

    Example:
        dropbox_upload_file('.', 'test.csv', '/stuff/test.csv')

    Returns:
        meta: The Dropbox file metadata.
    """

    try:
        dbx = dropbox_connect(at)

        local_file_path = pathlib.Path(local_path) / local_file

        with local_file_path.open("rb") as f:
            meta = dbx.files_upload(f.read(), dropbox_file_path, mode=dropbox.files.WriteMode("overwrite"))

            return meta
    except Exception as e:
        print('Error uploading file to Dropbox: ' + str(e))


In [55]:
def export2(b=None):
    try:
        selector_dict_day = load2.selector_dict_day
        raceEvents = load2.raceEvents 
        startDict = load2.startDict 
        log = load2.log
        filePath_export = top_pane_dict['file_choosers'][0].selected_path
        date_fileName = str(load2.log.TimeStamp.dropna()[1].date())
        fileName_export = "eventField_"+date_fileName
        events = functions.export_to_xml_worker(selector_dict_day, raceEvents, startDict, log, filePath_export, fileName_export)
        export2.events = events
        dropbox_upload_file(filePath_export, str(fileName_export)+'.xml', '/eventField_test/'+str(fileName_export)+'.xml',at)
        
        
    
    except:
        print("Failed to export XML file")


In [56]:
export_button = widgets.Button(description='Export Events as XML')
bottom_pane_output = widgets.Output()
   
export_button.on_click(export2)
display(export_button)



Button(description='Export Events as XML', style=ButtonStyle())

Event File exported as eventField_2022-05-20
